In [1]:
# Dataset
import os
if not os.path.exists('input.txt'):
    !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
len(text)  # 1M characters

1115394

In [4]:
text[:1000]  # First 1000 characters

"First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we become rakes: for the gods know I\nspeak this in hunger 

In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

print("".join(chars))
print(vocab_size)  # code book?


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


### Encode the string to a list of numbers

In [6]:
# create a mapping from characters to integers
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("Transformer"))
print(decode(encode("Transformer")))

[32, 56, 39, 52, 57, 44, 53, 56, 51, 43, 56]
Transformer


In [13]:
# another way of encoding
import tiktoken
# tiktok_enc = tiktoken.get_encoding('gpt2')
tiktok_enc.n_vocab

print(tiktok_enc.encode("Transformer"))
print(tiktok_enc.decode([8291, 16354]))

[8291, 16354]
Transformer


In [16]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)  # Encode and put the data into a tensor
data[:100]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])

In [17]:
# split the data into train and val

n = int(0.9*len(data)) # 9:1
train_data = data[:n]
val_data = data[n:]

In [22]:
block_size = 8  # context window size

train_data[:block_size+1]  # ordered --> make prediction on every character

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [39]:
import numpy as np

x = train_data[:block_size]
y = train_data[1:block_size+1]

for i in range(block_size):
    context = x[:i+1]
    target = y[i]
    print(f"context = {context.tolist()} -> target = {target}")

# Truncate the data so that it can be divided into blocks of block_size

context = [18] -> target = 47
context = [18, 47] -> target = 56
context = [18, 47, 56] -> target = 57
context = [18, 47, 56, 57] -> target = 58
context = [18, 47, 56, 57, 58] -> target = 1
context = [18, 47, 56, 57, 58, 1] -> target = 15
context = [18, 47, 56, 57, 58, 1, 15] -> target = 47
context = [18, 47, 56, 57, 58, 1, 15, 47] -> target = 58


In [41]:
torch.manual_seed(0)
batch_size = 4
block_size = 8 # maximum context length for predictions

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))  # random offset to training set
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb, '\n')
print('targets:')
print(yb.shape)
print(yb)

print('\n ---- \n')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[46, 43,  1, 45, 47, 60, 43, 57],
        [51,  1, 58, 46, 43,  1, 63, 53],
        [39, 50, 50,  1, 63, 53, 59,  1],
        [53, 61, 52,  1, 57, 53, 59, 50]]) 

targets:
torch.Size([4, 8])
tensor([[43,  1, 45, 47, 60, 43, 57,  1],
        [ 1, 58, 46, 43,  1, 63, 53, 59],
        [50, 50,  1, 63, 53, 59,  1, 46],
        [61, 52,  1, 57, 53, 59, 50,  1]])

 ---- 

when input is [46] the target: 43
when input is [46, 43] the target: 1
when input is [46, 43, 1] the target: 45
when input is [46, 43, 1, 45] the target: 47
when input is [46, 43, 1, 45, 47] the target: 60
when input is [46, 43, 1, 45, 47, 60] the target: 43
when input is [46, 43, 1, 45, 47, 60, 43] the target: 57
when input is [46, 43, 1, 45, 47, 60, 43, 57] the target: 1
when input is [51] the target: 1
when input is [51, 1] the target: 58
when input is [51, 1, 58] the target: 46
when input is [51, 1, 58, 46] the target: 43
when input is [51, 1, 58, 46, 43] the target: 1
when input is [5